In [12]:
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time

class Test(nn.Module):
    def __init__(self):
        super(Test, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1024, 64),
            nn.Linear(64, 100)
        )
 
    def forward(self,x):
        x = self.model(x)
        return x

In [ ]:
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time
#from model import *
 
# 定义训练设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 
# 准备数据集


train_data = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=torchvision.transforms.ToTensor())

test_data = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=torchvision.transforms.ToTensor())


train_data_size = len(train_data)
test_data_size = len(test_data)
# print("训练数据集的长度为{}".format(train_data_size))
# print("测试数据集的长度为{}".format(test_data_size))
 
# 利用DataLoader来加载数据集
train_dataloader = DataLoader(train_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

In [11]:
test = Test()
test = test.to(device)
 
# 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)
 
# 优化器
learning_rate = 0.01
optimizer = torch.optim.SGD(test.parameters(), lr=learning_rate)
 
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
epoch = 10
 
# 添加Tensorboard
writer = SummaryWriter("logs_train")

NameError: name 'Test' is not defined

In [12]:
def cutout(inputs, length=16):
    B, C, H, W = inputs.shape
    mask = np.ones((B, C, H, W), np.float32)
    for i in range(B):
        y = np.random.randint(H)
        x = np.random.randint(W)
        y1 = np.clip(y - length // 2, 0, H)
        y2 = np.clip(y + length // 2, 0, H)
        x1 = np.clip(x - length // 2, 0, W)
        x2 = np.clip(x + length // 2, 0, W)
        mask[i, :, y1:y2, x1:x2] = 0.
    inputs = inputs * torch.from_numpy(mask).to(inputs.device)
    return inputs


def mixup(image, labels, alpha):
    beta = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(image.size()[0])
    image_a = image
    labels_a = labels
    image_b = image[rand_index]
    labels_b = labels[rand_index]
    mixed_image = beta * image_a + (1 - beta) * image_b
    mixed_labels = beta * labels_a + (1 - beta) * labels_b
    return mixed_image, mixed_labels

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix(image, labels, alpha):
    beta = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(image.size()[0])
    image_a = image
    labels_a = labels
    image_b = image[rand_index]
    labels_b = labels[rand_index]
    bbx1, bby1, bbx2, bby2 = rand_bbox(image.size(), beta)
    image_a[:, bbx1:bbx2, bby1:bby2] = image_b[:, bbx1:bbx2, bby1:bby2]
    labels_a = labels_a * beta + labels_b * (1 - beta)
    labels_a = labels_a.type(torch.LongTensor)
    return image_a, labels_a

performance=[]

In [13]:
print('baseline')
start_time = time.time()
for i in range(epoch):
    print("-----第{}轮训练开始------".format(i+1))
 
    # 训练步骤开始
    test.train()
    for data in train_dataloader:
        imgs,targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        output = test(imgs)
        a=output
        b=targets
        loss = loss_fn(output, targets)
 
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
        total_train_step += 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time - start_time)
            print("训练次数{}, Loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
 
    # 测试步骤开始
    test.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            imgs=cutout(imgs)
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = test(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy
    performance.append(['baseline',total_test_loss,total_accuracy/test_data_size])
    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1
 
    torch.save(test, "baseline_test_{}.pth".format(i))
    print("模型已保存")
 
writer.close()

baseline


NameError: name 'time' is not defined

In [14]:
print('cutout')
start_time = time.time()
for i in range(epoch):
    print("-----第{}轮训练开始------".format(i+1))
 
    # 训练步骤开始
    test.train()
    for data in train_dataloader:
        imgs,targets = data
        imgs = imgs.to(device)
        imgs=cutout(imgs)
        targets = targets.to(device)
        output = test(imgs)
        a=output
        b=targets
        loss = loss_fn(output, targets)
 
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
        total_train_step += 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time - start_time)
            print("训练次数{}, Loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
 
    # 测试步骤开始
    test.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            imgs=cutout(imgs)
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = test(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy
    performance.append(['cutout',total_test_loss,total_accuracy/test_data_size])
    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1
 
    torch.save(test, "cutout_test_{}.pth".format(i))
    print("模型已保存")
 
writer.close()


cutout


NameError: name 'time' is not defined

In [15]:
print('mixup')
start_time = time.time()
for i in range(epoch):
    print("-----第{}轮训练开始------".format(i+1))
 
    # 训练步骤开始
    test.train()
    for data in train_dataloader:
        imgs,targets = data
        r = np.random.rand(1)
        alpha=1
        if alpha > 0 and r < 0.5:
            imgs,targets = mixup(imgs, targets, alpha)
        imgs = imgs.to(device)
        targets = targets.to(device)
        output = test(imgs)
        a=output
        b=targets
        loss = loss_fn(output, targets)
 
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
        total_train_step += 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time - start_time)
            print("训练次数{}, Loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
 
    # 测试步骤开始
    test.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            r = np.random.rand(1)
            alpha=1
            if alpha > 0 and r < 0.5:
                imgs,targets = mixup(imgs, targets, alpha)
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = test(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy
    performance.append(['baseline',total_test_loss,total_accuracy/test_data_size])
    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1
 
    torch.save(test, "mixup_test_{}.pth".format(i))
    print("模型已保存")
 
writer.close()

mixup


NameError: name 'time' is not defined

In [16]:
print('cutmix')
start_time = time.time()
for i in range(epoch):
    print("-----第{}轮训练开始------".format(i+1))
 
    # 训练步骤开始
    test.train()
    for data in train_dataloader:
        imgs,targets = data
        r = np.random.rand(1)
        alpha=1
        if alpha > 0 and r < 0.5:
            imgs,targets = cutmix(imgs, targets, alpha)
        imgs = imgs.to(device)
        targets = targets.to(device)
        output = test(imgs)
        a=output
        b=targets
        loss = loss_fn(output, targets)
 
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
        total_train_step += 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time - start_time)
            print("训练次数{}, Loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
 
    # 测试步骤开始
    test.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            r = np.random.rand(1)
            alpha=1
            if alpha > 0 and r < 0.5:
                imgs,targets = cutmix(imgs, targets, alpha)
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = test(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy
    performance.append(['baseline',total_test_loss,total_accuracy/test_data_size])
    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1
 
    torch.save(test, "cutmix_test_{}.pth".format(i))
    print("模型已保存")
 
writer.close()

cutmix


NameError: name 'time' is not defined

In [17]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages('output.pdf')
fig, axs = plt.subplots(3, 3)
row_titles = ['CutMix', 'Mixup', 'Cutout']
for photoindex in range(3):
    for i in range(3):
        dataiter = iter(train_dataloader)
        images, labels = next(dataiter)
        if i==0:
            images, labels = cutmix(images,labels,1)
        elif i==1:
            images, labels = mixup(images,labels,1)
        elif i==2:
            images=cutout(images)
        image=images[photoindex]
        image = image / 2 + 0.5  # unnormalize the image
        image = np.transpose(image, (1, 2, 0))
        axs[i,photoindex].imshow(image)
        axs[i,photoindex].axis('off')
        axs[photoindex, 0].set_title(row_titles[photoindex])
plt.show()
pdf.savefig(fig)

# Closethe PdfPages object
pdf.close()

ModuleNotFoundError: No module named 'matplotlib'